# Задание

Предложить модель программной реализации датчика измерения давления, температуры каждый пять минут, имеющего тип, диапазон эксплуационных характеристик

Может быть несколько датчиков этого типа в помещении.

Важно: класс ООП не использовать!

# Исследование

Рассмотрим характеристики датчиков, которые используются при работе с платами arduino и esp

BME280/BMP280

- Интерфейс: SPI, I2C;
- Напряжение питания: 3.3В;
- Диапазон измерений давления: 300-1100hPa;
- Диапазон измерений температуры: -40 - +85 °C;
- Диапазон измерений влажности: 0 - 100 %;
- Энергопотребление: режим измерений - 2.74 нА; в спящий режим: - 0.1 нА
- Точность измерений:давление - 0.01 hPa ( < 10 cm). Температура - 0.01° C Влажность – 3% 

DHT11

- Питание: DC 3,5 – 5,5 В
- Ток питания: в режиме измерения 0.3mA. В режиме ожидания 60μA
- определение влажности 20–80 % с точностью 5 %
- определение температуры 0–50 °С с точностью 2 %
- частота опроса не более 1 Гц (не более одного раза в 1 сек.)
- размеры 15,5´12´5,5 мм

BME680
- Датчик температуры:
    - рабочий диапазон: -40...+85 °С
    - абсолютная точность: ±1 °С
    - разрешение: 0,01 °С
    - потребление: 1 мкА при частоте измерений 1 Гц
- Датчик давления:
    - рабочий диапазон: 300...1100 кПа
    - разрешение: 0,18 Па
    - абсолютная погрешность: ±0,6 кПа
    - максимальная частота опроса: 182 Гц
    - типовое потребление: 3,1 мкА при частоте опроса 1 Гц
- Датчик влажности:
    - рабочий диапазон: 0...100%
    - абсолютная точность: ±3%
    - разрешение: 0,008%
    - максимальное потребление: 2,8 мкА при частоте опроса 1 Гц
    - типовое потребление: 2,1 мкА
    - старение: 0,5% в год

# Решения

Выделим общие характеристики
- Адресс драйвера датчика
- Тип датчика
- Минимальная температура (градусы)
- Максимальная температура (градусы)
- Погрешность температуры (градусы)
- Минимальное давление (Па)
- Максимальное давление (Па)
- Погрешность давления (Па)

Будем считать, что мы имеем доступ к драйверу serial порта датчика и драйверу для сохранения данных в базу данных

Предусмотрим место для буфера хранения значений измерений

In [13]:
import time
import datetime
from random import uniform

def millis():
    """ Эмулируем системный таймер """
    return round(time.time() * 1000)


def driver_get_measurement(obj):
    """ Получаем значения с датчика (случайные)"""
    temperature = uniform(obj['type']['min_t'], obj['type']['max_t'])
    pressure = uniform(obj['type']['min_p'], obj['type']['max_p'])
    return (temperature, pressure)
    
    
def driver_is_available(obj):
    """ Проверяем доступность датчика """
    return True
    
    
def database_send(obj):
    """ Сохраняем в базу данных значения, но здесь - выводим на экран """
    print(obj['address'], obj['type']['type_name'], ':', *obj['buffer'])
    obj['buffer'] = []
    
    
def database_is_available():
    """ Проверяем доступность базы данных """
    return True

    
def create_type(type_name: str, 
           min_t: float, max_t: float, acc_t: float,
           min_p: float, max_p: float, acc_p: float):
    """ Создание объекта-словаря для типа датчика """
    return {"type_name": type_name,
           "min_t": min_t, "max_t": max_t, "acc_t": acc_t,
           "min_p": min_p, "max_p": max_p, "acc_p": acc_p,}
    
    
def create_sensor(address: int, type_s):
    """ Создание объекта-словаря для датчика """
    return {"address": address, "type": type_s, "buffer": []}
    
    
# Описание типов датчиков
st_bme280 = create_type(
    type_name="bme280", 
    min_t=-40.0, max_t=85.0, acc_t=0.01,
    min_p=300_000, max_p=1_000_000, acc_p=10
)

st_bme680 = create_type(
    type_name="bme680", 
    min_t=-40.0, max_t=85.0, acc_t=1,
    min_p=300_000, max_p=1_000_000, acc_p=180
)

# Описание датчиков в наличии
sensors = [
    create_sensor(100001, st_bme280), 
    create_sensor(100002, st_bme680),
    create_sensor(100003, st_bme280), 
    create_sensor(100004, st_bme680)
]

# Таймер
timer = 0 
MY_PERIOD = 5 * 1000 

# Рабочий цикл
k = 3 # -1 для бесконечной работы
while True:
    
    if millis() - timer >= MY_PERIOD:
        timer = millis()  # сброс таймера
        print(datetime.datetime.now()) # выводим время периода измерения
        
        for sensor in sensors:
            if driver_is_available(sensor):
                meas = driver_get_measurement(sensor)
                sensor['buffer'].append(meas)
                
            if database_is_available():
                database_send(sensor)
                
        k -= 1
        if k == 0:
            break



2024-10-25 22:12:59.232689
100001 bme280 : (41.43953227493225, 478122.46382051456)
100002 bme680 : (34.89907597224516, 758372.0162381517)
100003 bme280 : (-19.630784856821784, 544798.5898225061)
100004 bme680 : (15.151615509160202, 710394.4914977201)
2024-10-25 22:13:04.232582
100001 bme280 : (-32.8160070716997, 342878.441444426)
100002 bme680 : (64.89487911367226, 330659.2098744276)
100003 bme280 : (33.05229899623339, 658337.6473858793)
100004 bme680 : (-9.201721760295108, 544014.4123399686)
2024-10-25 22:13:09.232865
100001 bme280 : (-17.918383274700943, 925711.6114235099)
100002 bme680 : (72.79687763716164, 397941.39564284426)
100003 bme280 : (81.86099200171638, 347876.7576289467)
100004 bme680 : (-4.539428006718765, 721915.1180406576)
